# Large Language Model Prompt Engineering

***
In this lab, we will cover what is prompt engineering, what are some real world use cases, and advanced prompt engineering insights.
***

1. [Set Up](#1.-Set-Up)
2. [N-shots Prompting Concepts](#2.-run-inference-on-the-pre-trained-model)
3. [Chain of Thought (CoT) Prompting](#3.-Query-endpoint-and-parse-response)
4. [Tree of Thought (ToT) Prompting](#4.-Use-Cases)

## 1. Set Up

The endpoint URL is pointing to a AI21 Jurassic 2 Grande Instruct model deployed on Amazon Sagemaker endpoint. This will be the model used for this lab.

In [3]:
import logging
import json
import time
import boto3

import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm.contrib.concurrent import thread_map

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

url = "https://e165qoav65.execute-api.us-east-1.amazonaws.com/test/"

In [4]:
# Define a method that will invoke the REST API via HTTP request to interact with the model. The returned response will be the
# text generated by the model.
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    
    response_body = response.content
    response_json = json.loads(response_body.decode('utf-8'))
    generated_text = response_json['completions'][0]['data']['text']
    return generated_text

### Model Parameters

***
AI21 Jurassic 2 Models supports many advanced parameters while performing inference. Here are some commonly used parameters:

* **maxTokens:** The maximum number of tokens to generate per result. Optional, default = 16. If no stopSequences are given, generation is stopped after producing maxTokens
* **numResults:** Number of completions to sample and return. Optional, default = 1
* **temperature:** Modifies the distribution from which tokens are sampled. Optional, default = 0.7 Setting temperature to 1.0 samples directly from the model distribution. Lower (higher) values increase the chance of sampling higher (lower) probability tokens. A value of 0 essentially disables sampling and results in greedy decoding, where the most likely token is chosen at every step. ​
* **topKReturn:** 0 <= integer <= 10, Optional, default = 0. Return the top-K alternative tokens. When using a non-zero value, the response includes the string representations and logprobs for each of the top-K alternatives at each position, in the prompt and in the completions.
* **topP:** Sample tokens from the corresponding top percentile of probability mass. Optional, default = 1. For example, a value of 0.9 will only consider tokens comprising the top 90% probability mass.
* **stopSequences:** Stops decoding if any of the strings is generated. Optional. For example, to stop at a comma or a new line use [".", "\n"]. The decoded result text will not include the stop sequence string, but it will be included in the raw token data, which can also continue beyond the stop sequence if the sequence ended in the middle of a token. The sequence which triggered the termination will be included in the finishReason of the response.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

## 2. N-shots Prompting Concepts

There are some prompting techniques that we can make use of to perform tasks more effectively with LLM. In this section, we will cover what is zero-shot, one-shot, and few-shots prompting. This is also known as N-shots prompting. While walking through these concepts, we will also introduce what are some real world applications of LLM.
***

### 2.1 Zero Shot Prompting 

Zero shot prompting refers to prompting the model to generate predictions on unseen data without the need for any additional training. This can be used for more straightforward tasks as shown below.
***

#### 2.1.1 In Context Question & Answering
LLMs are able to make inferences based on the context of the input text. This is done through a technique called "attention," which allows the model to focus on specific parts of the input text when making predictions. For example, if you input a document about a particular topic, the model can infer the main points of the document and answer follow-up questions about the topic. This allows you to interact with and draw insights from documents, transcripts, articles etc.

##### *Document QnA*

In [27]:
context = """Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a branch. \
The fox devised a plan and flattered the crow, causing the crow to caw with delight, dropping the \
cheese which the fox quickly snatched up and ran away. The crow learned a valuable lesson and never trusted the fox again."""

question = 'Who got cheated?'
question1 = 'How did the fox cheat the crow?'
question2 = 'what is the lesson from this story?'

'''
Change the question variable here to test out the other questions. Feel free to ask your own questions and see what will the 
answer be!
'''
prompt = f'Context: {context}<br>Question: {question1}<br>Answer:'
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Context: Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a branch. The fox devised a plan and flattered the crow, causing the crow to caw with delight, dropping the cheese which the fox quickly snatched up and ran away. The crow learned a valuable lesson and never trusted the fox again.<br>Question: How did the fox cheat the crow?<br>Answer:


In [174]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)

logger.info(f'{response}')

 
The original price of the gallon of milk is $3.99.
The discount is 25%, so 3.99/100 * 25 = $0.99.
Discounted price is $3.99-$0.99=$3.00.
$1 off the entire purchase is $3.00-$1=$2.00.
The final price of the gallon of milk after all the discount is $2.00.


##### *Transcript QnA*

In [71]:
context = """
Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try restarting your iPhone. Press and hold the power button until you see the "slide to power off" option. Slide to power off, wait a few seconds, and then turn your iPhone back on.
Customer: Alright, I restarted it, but it's still not charging properly.
Agent: I see. It looks like we need to run a diagnostic test on your iPhone. Please visit the nearest Apple Store or authorized service provider to get your iPhone checked out.
Customer: Do I need to make an appointment?
Agent: Yes, it's always best to make an appointment beforehand so you don't have to wait in line. You can make an appointment online or by calling the Apple Store or authorized service provider.
Customer: Okay, will I have to pay for the repairs?
Agent: That depends on whether your iPhone is covered under warranty or not. If it is, you won't have to pay anything. However, if it's not covered under warranty, you will have to pay for the repairs.
Customer: How long will it take to get my iPhone back?
Agent: It depends on the severity of the issue, but it usually takes 1-2 business days.
Customer: Can I track the repair status online?
Agent: Yes, you can track the repair status online or by calling the Apple Store or authorized service provider.
Customer: Alright, thanks for your help.
Agent: No problem, happy to help. Is there anything else I can assist you with?
Customer: No, that's all for now.
Agent: Alright, have a great day and good luck with your iPhone!
"""

query = (
    "What are the customer and agent talking about?"
)

# "What troubleshooting steps were suggested to the customer to fix their iPhone charging issue?"
# "Was resetting the iPhone to its default settings able to solve the charging issue and battery draining?"
# "What steps can the customer take to make an appointment at the nearest Apple Store or authorized service provider?"
# "What is the overall sentiment and sentiment score of the conversation between the customer and the agent?"
# "identify any specific words, phrases, or context that influenced the {sentiment} sentiment."

prompt = f"""
Context: 
{context}

{query}
"""

logger.info(f'Prompt: {prompt}')

Prompt: 
Context: 

Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try r

In [73]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

The customer is having a problem with their iPhone, and the agent is helping them troubleshoot the issue. The customer is not charging properly, and the battery is draining quickly. The agent suggests force quitting some apps, resetting the iPhone's settings, restarting it, and running diagnostic tests. The customer may have to pay for the repairs if the iPhone is not under warranty.


***
#### 2.1.2 Multi-Class Classification

LLMs have natural language understanding capabilities. Given an input, they are able to infer the meaning of the texts and classify them into the categories you define. Below are some possible uses cases.

##### *Sentiment Analysis*

In [56]:
review = 'I hated the movie. Thoroughly disappointing for a sequel.'
review1 = 'The movie is alright. Will consider watching it again'
review2 = 'What an awesome movie! Excited for the next sequel to be out.'

prompt = f'Analyze the sentiment of the following review:\n{review}\nClassify the sentiment as either one of the following [positive, neutral, negative].\nSentiment:'
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Analyze the sentiment of the following review:
I hated the movie. Thoroughly disappointing for a sequel.
Classify the sentiment as either one of the following [positive, neutral, negative].
Sentiment:


In [55]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

 negative


&nbsp;
##### *Topic Modelling*

In [64]:
article = 'Penguins are flightless birds that live in the Southern Hemisphere. They are excellent swimmers and divers, and their diet consists of fish, squid, and krill. Penguins are social animals, and they live in colonies. They are known for their waddling gait and their black and white feathers, which help them to camouflage themselves from predators.'
article1 = 'Laksa is a spicy noodle soup popular in Southeast Asia. It is typically made with a rich and flavorful coconut milk broth, rice noodles, seafood, and a variety of herbs and spices. Laksa is a hearty and satisfying dish that is sure to warm you up on a cold day.'
article2 = 'A car is a four-wheeled motor vehicle that is designed to carry passengers. Cars are powered by an internal combustion engine and can travel at speeds of up to 100 miles per hour. Cars are a popular mode of transportation for both personal and commercial use.'

prompt = f'Analyze the following article:\n{article2}\nClassify the article as either one of the following topics [Animal, Food, Vehicle].\nTopic:'
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Analyze the following article:
A car is a four-wheeled motor vehicle that is designed to carry passengers. Cars are powered by an internal combustion engine and can travel at speeds of up to 100 miles per hour. Cars are a popular mode of transportation for both personal and commercial use.
Classify the article as either one of the following topics [Animal, Food, Vehicle].
Topic:


In [65]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

 Vehicle


#### 2.1.3 Natural Language Inference (NLI)

In [66]:
location = "United States"
# location = "United Kingdom"
prompt = f"""The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in {location}"?
"""
logger.info(f"Prompt: {prompt}")

Prompt: The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in United States"?



In [68]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
print(response)

No, it is not possible to conclude from the paragraph that the world cup takes place in United States. The paragraph only mentions that the world cup has kicked off in Los Angeles, United States. It does not provide any information about the location or host country of the world cup.


***
#### 2.1.4 Text Generation
The ability of LLMs to generate text is one of its greatest strength. This is an area that is under continuous research. Based on the many experiements and research in the current landscape, here are some tips and ideas on how you can leverage this capability on real world use cases.

##### *Generating a product description*
When generatiing creative write-ups like product description, setting a higher temperature gives the LLM some space to be creative. This way, you can generate multiple outputs and choose 1 that suits you the most! The more details you provide, the more accurate the generated description will be.

In [85]:
prompt = """Product features:
- A bottle that can keep your hot drinks hot for up to 12 hours, cold drinks cold for up to 8 hours
- Small and can fit into most bags easily, making it easy to carry around
- Modern woody designs, making it aesthetically pleasing

Based on the above product features, help to write a product description that will attract consumers to purchase the bottle. Include some marketing gimmicks.
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Product features:
- A bottle that can keep your hot drinks hot for up to 12 hours, cold drinks cold for up to 8 hours
- Small and can fit into most bags easily, making it easy to carry around
- Modern woody designs, making it aesthetically pleasing

Based on the above product features, help to write a product description that will attract consumers to purchase the bottle. Include some marketing gimmicks.



In [87]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0.7}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

This bottle is perfect for those who want to enjoy a hot drink on the go. It's small and can fit in most bags, so you can easily take it with you. The modern design is also aesthetically pleasing. Plus, the insulation will keep your drinks hot for up to 12 hours or cold for up to 8 hours.


&nbsp;
##### *Editing texts to different styles*
LLMs are trained on large corpus of texts and have the ability to generate texts in writing styles. Here, we explore the following styles that LLM are able to generate for us:
* Changing the type of narrative
* Adapting to different domain
* Constructing an email

There are many more possible styles that it can generate, feel free to give it any style you can think of!

In [110]:
style = 'first person narrative'
style1 = 'travel agency marketing campaign'
style2 = 'business formal email'
prompt = f"""Write-up:
Singapore is a city-state located in Southeast Asia. It is a small country, only about 710 square kilometers in size, but it is very densely populated. The population of Singapore is about 5 million people, and it is made up of people from many different cultures, including Chinese, Malay, Indian, and Eurasian.

Re-write the above write-up as a {style}.
"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Write-up:
Singapore is a city-state located in Southeast Asia. It is a small country, only about 710 square kilometers in size, but it is very densely populated. The population of Singapore is about 5 million people, and it is made up of people from many different cultures, including Chinese, Malay, Indian, and Eurasian.

Re-write the above write-up as a business formal email.



In [111]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

Dear Sir/Madam,

I am writing to you today to inform you about Singapore. Singapore is a city-state located in Southeast Asia. It is a small country, only about 710 square kilometers in size, but it is very densely populated. The population of Singapore is about 5 million people, and it is made up of people from many different cultures, including Chinese, Malay, Indian, and Eurasian.

Singapore is known for its clean and efficient government, as well as its strong economy. The country has a highly developed infrastructure, and is known for its excellent public transportation system. Singapore is also known for its many tourist attractions, including the famous Singapore Botanic Gardens, the Singapore Zoo, and the Singapore River Cruise.

If you are interested in visiting Singapore, I would be happy to provide you with more information. I look forward to hearing from you soon.


#### 2.1.5 Translation

In [ ]:
prompt = "Translate to German:  My name is Arthur"

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

In [ ]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 0.2, "do_sample": False, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### In Context Q&A
Move to document QnA

In [ ]:
test_context = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) \
were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. \
They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, \
Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. \
Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, \
their descendants would gradually merge with the Carolingian-based cultures of West Francia. \
The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, \
and it continued to evolve over the succeeding centuries."""
test_question = "In what country is Normandy located?"

prompt = f"""
Context: 
{test_context}

{test_question}
"""

logger.info(f'Prompt: {prompt}')

In [ ]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 1.2, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

### 2.2 One-shot / Few shot Learning
Few shot learning is useful when you want the generated text to follow specific structures, tone, or choice of words. It works by providing a one or few examples within the prompt. The LLM will be able to learn from the examples and generate similar output. This is also known as In-Context Learning.

#### Text Summarisation
Move to zero shot

In [ ]:
train_article = 'I love apples especially the large juicy ones. Apples are a great source of vitamins and fiber. An apple a day keeps the doctor away!'
train_summary = 'I love apples.'

test_article = 'I hate oranges especially the bitter ones. They are high in citric acid and they give me heart burns. Doctor suggests me to avoid them!'
test_summary = 'I hate oranges.'

In [ ]:
prompt = f"""
article: {train_article}
summary: {train_summary}
--
article: {test_article}
summary:
"""

logger.info(f'Prompt: {prompt}')

In [ ]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": 0.5, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### Natural Language Generation (NLG)
Move to text generation

In [ ]:
train_inp = 'name[The Punter], eat_type[Indian], price_range[cheap]'
train_out = 'The Punter provides Indian food in the cheap price range.'

test_inp = 'name[Blue Spice], eatType[coffee shop], price_range[expensive]'
test_out = 'Blue Spice is a coffee shop that is a bit expensive.'

prompt = (
    f"{train_inp} ==> {train_out}\n"
    f"--\n"
    f"{test_inp} ==>"
)

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

In [ ]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 0.5, "seed": 123, "num_beams": 3}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### 2.2.1 Entity Extraction
We can perform entity extraction by providing examples to the LLM. The LLM will follow the stucture in the provided examples when generating the output.

In [114]:
train_inp = 'The Punter provides Indian food in the cheap price range.'
train_out = 'name[The Punter], eat_type[Indian], price_range[cheap]'

test_inp = 'Blue Spice is a coffee shop that is a bit pricy.'
test_out = 'name[Blue Spice], eat_type[coffee shop], price_range[pricy]'

prompt = (
    f"""{train_inp} ==> {train_out}\n"""
    f"--\n"
    f"""{test_inp} ==>"""
)

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt: The Punter provides Indian food in the cheap price range. ==> name[The Punter], eat_type[Indian], price_range[cheap]
--
Blue Spice is a coffee shop that is a bit pricy. ==>


In [115]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

 name[Blue Spice], eat_type[coffee shop], price_range[pricy]


***
#### 2.2.2 Natural Language Query
Given information on the database schema, LLMs can generate the SQL query required to extract the information you have requested for. Here is an example on how the LLM can generate SQL queries based on a question asked in natural language. A few examples is also shown to ensure that the generated output follows a fixed structure.

In [145]:
prompt = """Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: all the countries we have customers in without repetitions.
SQL statement:
SELECT DISTINCT Country FROM Customers;

##

Create SQL statement from instruction.

Database: Products(ProductID, ProductName, SupplierID, CategoryID, Unit, Price)
Request: selects all products from categories 1 and 7
SQL statement:
SELECT * FROM Products
WHERE CategoryID = 1 OR CategoryID = 7;

##

Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: there is a customer who lives in Frankfurt city. change the customer's name to Alfred Schmidt.
SQL statement:"""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: all the countries we have customers in without repetitions.
SQL statement:
SELECT DISTINCT Country FROM Customers;

##

Create SQL statement from instruction.

Database: Products(ProductID, ProductName, SupplierID, CategoryID, Unit, Price)
Request: selects all products from categories 1 and 7
SQL statement:
SELECT * FROM Products
WHERE CategoryID = 1 OR CategoryID = 7;

##

Create SQL statement from instruction.

Database: Customers(CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
Request: there is a customer who lives in Frankfurt city. change the customer's name to Alfred Schmidt.
SQL statement:


In [146]:
payload = {"prompt": prompt, "maxTokens": 200, "temperature": 0, "stopSequences": ["##"]}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')


UPDATE Customers
SET CustomerName = 'Alfred Schmidt'
WHERE City = 'Frankfurt';


#### Entity Extraction

In [ ]:
prompt = f"""
Extract main person:

s: John is playing basketball
p: John
##
s: Jeff and Phil are chatting about GAI. Phil has to run. He is in a rush
p: Phil
##
s: Max is older than Emma
p: Max
##
s: Susan misses the bus this morning but still get in time for her meeting with Sara
p:
"""

In [ ]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### Classification

In [ ]:
prompt = f"""
Classify the topic of the following paragraph: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Label: Business.

##

Classify the topic of the following paragraph: Some People Not Eligible to Get in on Google IPO Google has billed its IPO as a way for everyday people to get in on the process, denying Wall Street the usual stranglehold it's had on IPOs. Public bidding, a minimum of just five shares, an open process with 28 underwriters - all this pointed to a new level of public participation. But this isn't the case.
Label: Technology.

##

Classify the topic of the following paragraph: Indians Mount Charge The Cleveland Indians pulled within one game of the AL Central lead by beating the Minnesota Twins, 7-1, Saturday night with home runs by Travis Hafner and Victor Martinez.
Label: Sports.

##

Classify the topic of the following paragraph: Uptown girl, she's been living in her uptown world, I bet she never had a backstreet guy, I bet her mother never told her why, I'm gonna try.
Label:
"""

In [ ]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

In [ ]:
prompt = f"""

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: My native language is not English, I have blogs and I write my own articles. I also get articles from outsource writers, so I want to use it to re-write such articles, so i will create a tool for that
NLP Task: Paraphrasing
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Just experimenting with content generation
NLP task: Long form generation
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: My company MetaDialog provides human in the loop automated support for costumers, we are currently using GPT3 to generate answers using our custom search engine to clients questions, and then provide the answers as suggestions to human agents to use or reject them as real answers to clients.
NLP task: Conversational agent
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Receipt extraction including line items from plain text (sources being OCR-ed images, PDFs and HTML Emails)
NLP task: Information extraction
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: I have a lot of legacy documentation which needs to be cleaned, summarized, and queried. I think AI21 would help.
NLP task: Summarization

##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Answer questions based on a given corpus of information
NLP task: Question answering
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: creating useful content for companies websites articles
NLP task:
"""

In [ ]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### Fraud Detection
Provide a few sample emails that are legitimate and fraudulent. The LLM will learn from the examples and use the information to determine if an email is fraudulent.

***
## 3. Chain of Thought (CoT) Prompting
Chain of thought prompting is a method for improving the reasoning abilities of large language models. It works by providing the model with a few examples of how to solve a problem step-by-step. This helps the model to learn how to break down complex problems into smaller, more manageable tasks.

In [172]:
prompt = """Question: You are shopping at a store and you see a shirt that you like. The shirt is originally priced at $20, but it is on sale for 20% off. You also have a coupon that get you $4 off of your entire purchase. How much will you pay for the shirt with all the discounts? Solve the problem step by step.
Answer: 
The original price of the shirt is $20.
The discount is 20%, so 20/100 * 20 = $4.
Discounted price is $20-$4=$16.
$4 off the entire purchase is $16-$4=$12.
The final price of the shirt after all the discount is $12.

Question: You are shopping at a grocery store and you see a gallon of milk that is originally priced at $3.99. The milk is on sale for 25% off. You also have a coupon for $1 off of your entire purchase. How much will you pay for the milk? Solve the problem step by step.
Answer:"""


logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Question: You are shopping at a store and you see a shirt that you like. The shirt is originally priced at $20, but it is on sale for 20% off. You also have a coupon that get you $4 off of your entire purchase. How much will you pay for the shirt with all the discounts? Solve the problem step by step.
Answer: 
The original price of the shirt is $20.
The discount is 20%, so 20/100 * 20 = $4.
Discounted price is $20-$4=$16.
$4 off the entire purchase is $16-$4=$12.
The final price of the shirt after all the discount is $12.

Question: You are shopping at a grocery store and you see a gallon of milk that is originally priced at $3.99. The milk is on sale for 25% off. You also have a coupon for $1 off of your entire purchase. How much will you pay for the milk? Solve the problem step by step.
Answer:


In [173]:
payload = {"prompt": prompt, "maxTokens": 300, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')

 
The original price of the gallon of milk is $3.99.
The discount is 25%, so 3.99/100 * 25 = $0.99.
Discounted price is $3.99-$0.99=$3.00.
$1 off the entire purchase is $3.00-$1=$2.00.
The final price of the gallon of milk after all the discount is $2.00.


In the above example, we have looked at how LLMs can learn from In-Context examples and perform multi-step reasoning to solve a mathematical problem. Here, we look at how we can explicitly instruct the LLM to solve the problem 'step by step' to improve its logical reasoning capability and derive the right answer.

In [166]:
prompt = """You are planning a party and you need to buy food and drinks for 20 people. Each person requires 3 cup of drinks, and 3 slices of pizza. 1 cup of drink cost $2. 1 slice of pizza cosr $3. How much will the total food and drinks cost for the party?
Solve this problem step by step."""

logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
You are planning a party and you need to buy food and drinks for 20 people. Each person requires 3 cup of drinks, and 3 slices of pizza. 1 cup of drink cost $2. 1 slice of pizza cosr $3. How much will the total food and drinks cost for the party?
Solve this problem step by step.


In [167]:
payload = {"prompt": prompt, "maxTokens": 300, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)
logger.info(f'{response}')


1. First find the cost of 3 cups of drinks: 3 x $2 = $6
2. Then find the cost of 3 slices of pizza: 3 x $3 = $9
3. Then add the cost of 3 cups of drinks and 3 slices of pizza to find the total cost: $6 + $9 = $15
4. Then multiply the total cost by 20 to find the total cost for 20 people: $15 x 20 = $300

Therefore, the total cost for food and drinks for 20 people is $300.


## 4. Tree of Thoughts (ToT) Prompting